In [ ]:
from promptflow.client import PFClient
import json

## Evaluate an LLM

### Evaluate Locally with Azure Promptflow

In [ ]:
# PFClient can help manage your runs and connections.
pf = PFClient()

# Define Flows and Data
chat_flow = "./src/promptflows/chat_rag_wiki" # set the flow directory
eval_flow = "./src/promptflows/eval_flow" # set flow directory
data = "./data/input_test_data.csv" # set the data file

# Run chat flow to generate chat results
rag_chat_run = pf.run(
    flow=chat_flow,
    data=data,
    stream=False,
    column_mapping={  # map the url field from the data to the url input of the flow
      "input": "${data.input}",
    }
)

# Run evaluation flow to evaluate chat results
rag_eval_run = pf.run(
    flow=eval_flow,
    data=data,
    run=rag_chat_run,
    stream=False,
    column_mapping={  # map the url field from the data to the url input of the flow
      "question": "${data.input}",
      "answer": "${run.outputs.answer}",
      "context": "${run.outputs.context}"
    }
)

In [8]:
display(pf.get_details(rag_eval_run))

display(json.dumps(pf.get_metrics(rag_eval_run)))

,inputs.question,inputs.answer,inputs.context,inputs.line_number,outputs.output
0,Who was Joan of Arc?,"Joan of Arc was a patron saint of France, know...",Joan of Arc French Jeanne dArc an dak Middle F...,0,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
1,Who was Saladin?,"Saladin, born Salah ad-Din Yusuf ibn Ayyub, wa...",Salah adDin Yusuf ibn Ayyuba c1137 4 March 11...,1,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
2,Who was Alexander the great?,Alexander the Great was a king of the ancient ...,Alexander III of Macedon Ancient Greek romani...,2,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
3,Who was King Arthur?,King Arthur is a legendary king of Britain who...,King Arthur Welsh Brenin Arthur Cornish Arthur...,3,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
4,Who was Leonardo da Vinci?,Leonardo da Vinci was an Italian polymath acti...,Leonardo di ser Piero da Vincib 15 April 14522...,4,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
5,What was the Renaissance?,The Renaissance was a cultural movement from t...,The Renaissance UK rnesns rnAYsnss US rnsns R...,5,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
6,Who was Cleopatra?,Cleopatra VII Thea Philopator was the Queen of...,Cleopatra VII Thea Philopator Koin Greek not...,6,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
7,What was the Industrial Revolution?,The Industrial Revolution was a global transit...,The Industrial Revolution sometimes divided in...,7,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
8,Who was Nelson Mandela?,Nelson Mandela was a South African anti-aparth...,Nelson Rolihlahla Mandela mndl manDEHl1 Xhosa ...,8,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."
9,What was the significance of the moon landing ...,"The significance of the moon landing in 1969, ...",Apollo 11 July 1624 1969 was the American spac...,9,"{'fluency': 5, 'groundedness': 5, 'answer_cont..."


'{"fluency": 4.933333333333334, "groundedness": 5, "answer_context_sim": 0.95}'

In [9]:
metric_dict = dict(pf.get_metrics(rag_eval_run))
metric_dict['fluency']

4.933333333333334

### Batch Endpoint Evaluation

### 